In [6]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

In [8]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_transformers import LongContextReorder

In [9]:
texts = [
    "이건 그냥 내가 아무렇게나 적어본 글입니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]

In [10]:
def format_docs(docs):
    result = []

    for i, doc in enumerate(docs):
        result.append(f"[{i}] {doc.page_content} [source: yoogane_kmu@naver.com]")
    
    return "\n".join(result)

In [11]:
def reorder_documents(docs):            # 재정렬    
    reordering = LongContextReorder()           # LongContextReorder` 클래스의 인스턴스인 reordering을 생성
    reordered_docs = reordering.transform_documents(docs)
    combined = format_docs(reordered_docs)
    print(combined)
    
    return combined

In [12]:
# 프롬프트 템플릿
template = """Given this text extracts:
{context}

-----
Please answer the following question:
{question}

Answer in the following languages: {language}
"""

In [13]:
# 프롬프트 정의
prompt = ChatPromptTemplate.from_template(template)

In [14]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 검색기를 생성합니다.
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

In [15]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [19]:
# Chain 정의
chain = (
    {
        "context": itemgetter("question")       # "ChatGPT에 대해 무엇을 말해줄 수 있나요?" 를 retriever에 던진다.
        | retriever                             # retriever는 질문에 대한 검색하게 된다 (결과는 내림차순 정렬의 문서)
        | RunnableLambda(reorder_documents),    # 위의 결과를 reorder_documents() 메서드로 넣어서 실행. 순위 재조정이 일어난다. 결과는 'context'에 들어간다.
        "question": itemgetter("question"),     # 질문("ChatGPT에 대해 무엇을 말해줄 수 있나요?")을 추출합니다.
        "language": itemgetter("language"),     # 답변 언어를 추출합니다. 위의 3가지 모두가 prompt로 들어갑니다.
    }                                           
    | prompt                                    # 프롬프트
    | llm                                       # LLM
    | StrOutputParser()                         # 출력파서
)

In [20]:
answer = chain.invoke(
    {"question": "ChatGPT에 대해 무엇을 말해줄 수 있나요?", "language": "KOREAN"}
)

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: yoogane_kmu@naver.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: yoogane_kmu@naver.com]
[2] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: yoogane_kmu@naver.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: yoogane_kmu@naver.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: yoogane_kmu@naver.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: yoogane_kmu@naver.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: yoogane_kmu@naver.com]
[7] 이건 그냥 내가 아무렇게나 적어본 글입니다. [source: yoogane_kmu@naver.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: yoogane_kmu@naver.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: yoogane_kmu@naver.com]


In [21]:
print(answer)

ChatGPT는 OpenAI에 의해 개발된 인공지능으로, 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. 이 AI는 지속적인 학습과 업데이트를 통해 발전하고 있으며, 다양한 질문에 답할 수 있도록 설계되었습니다. 또한, 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
